**Proyecto Final**

Robotica Industrial

Carlos Andres Uriegas Garcia de Alba

Michel Emanuel López Franco

25 de abril de 2020


En este proyecto se presenta una manera de trabajar deteccion y reconocimiento facial a traves de la camara de la computadora en Google Colab. Como el objetivo del proyecto es tanto la transmisión de video como la detección de rostros dentro de un navegador web, se utilizo una combinacion de Python y Javascript. El programa toma un video del cual captura imagenes y las procesa para entrenar con ellas y asociar un nombre a la persona en las imagenes. Despues el programa toma un video de prueba en el que se busca identificar si la persona en el video correpsonde a alguna persona previamente registrada a traves del entrenamiento. Si es asi, el programa presenta una imagen del video de prueba en la que un recuadro enmarca la cara de la persona y un texto en el recuadro indica el nombre de la persona enmarcada. 

Antes de comenzar, se debe de descargar un zip con el nombre "face-detection-master.zip" que se encuentra en la siguiente liga https://drive.google.com/open?id=1Ox3B9MGAWOOnMLVyzXGBGUmMZFL2i_R3 .
Despues descomprima el archivo "face-detection-master.zip", copie la carpeta "Face-Detection-Master" en su Google Drive y cambie el nombre de la carpeta a "Colab Notebooks". 
Si es necesario, cambie el nombre de la subcarpeta llamada "Carlos" (mi nombre) y cambiela en el codigo. Esta es la carpeta que almacena las imágenes .png (primeros 30 cuadros) extraidas del video de entrenamiento

Las primeras dos lineas de codigo envian al usuario a confirmar que el programa puede escribir y leer archivos de su cuenta de Google Drive. Ahi se guardaran los videos capturados y los archivos necesarios para el programa. Esto es necesario para mantener el programa totalmente en linea. 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Las siguientes lineas de codigo son una combinacion de Python y Javascript que inicializa variables globales, administra la camara de la computadora, prepara las capturas de video, conecta la transmision de la camara a un html, y la muestra en la pantalla.

In [0]:
!pip install ffmpeg-python


from IPython.display import HTML, Javascript, display
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import io
import ffmpeg

video_file_train = '/content/drive/My Drive/Colab Notebooks/Video Datasets/osy_train.mp4' 
video_file_test = '/content/drive/My Drive/Colab Notebooks/Video Datasets/osy_test.mp4' 
  

VIDEO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var my_btn_txt = document.createTextNode("Press to start recording");

my_btn.appendChild(my_btn_txt);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, videoStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  videoStream = stream;
  var options = {  
    mimeType : 'video/webm;codecs=vp9'  
  };            
  recorder = new MediaRecorder(stream, options);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('video');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({video: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      videoStream.getVideoTracks()[0].stop();
      recordButton.innerText = "Saving the recording... Please wait!"
  }
}

function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def start_webcam():
  js = Javascript('''
    async function startWebcam() {
      const div = document.createElement('div');

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      
      return;

    }
    ''')
  
  display(js)
  data = eval_js('startWebcam()')
  
    
start_webcam()

def get_video():
  display(HTML(VIDEO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  return binary

Las siguientes lineas de codigo inician la grabacion de la transmisión de video de la camara y la convierten en datos binarios a traves de la funcion *get_video()* definida previemnte. Una vez que se presiona el boton para detener la grabacion, el programa retorna los datos binarios y se guardan como un archivo .mp4. que es almacenado en la carpeta llamada "Video Dataset" del Google Drive asociado previamente.

In [0]:
videofile = get_video()

with open(video_file_train, 'wb') as f:
  f.write(videofile)

Las siguientes lineas de codigo son las mismas que las anteriores, la diferencia es que ahora graban el video que sera usado para la deteccion facial. Es decir, la cara que se identifique en este video sera comparada con la cara previamente almacenada en las lineas anteriores. 

In [0]:
videofile = get_video()

with open(video_file_test, 'wb') as f:
  f.write(videofile)

Las siguientes lineas de codigo hacen uso de los videos de entrenamiento y de prueba. 

Primero se extraen los primeros 30 cuadros del video de entrenamiento y se almacenan como imagenes .png en la carpeta llamada "Image Dataset" del Google Drive asociado previamente. Cada imagen se convertira de color a una escala de grises. Luego, las caras seran pre-detectadas por un algoritmo OpenCV, *CascadeClassifier.detectMultiScale()*. Este algoritmo permite detectar objetos y devuelve una lista de rectángulos. OpenCV usa un clasificador para detectar objetos particulares llamado *haarcascade_frontalface_default.xml*. El *haarcascade_frontalface_default.xml* es un archivo xml haar cascade diseñado por OpenCV para detectar la cara frontal. Finalmente, las imagenes se redimensionan a un formato mas pequeño y se almacenan en la carpeta "Image Dataset / Carlos" de Google Drive (puede cambiar en el codigo el nombre "Carlos" a el nombre con el que desee asociar las imagenes).

In [0]:
import cv2, sys, numpy, os
from time import sleep
from google.colab.patches import cv2_imshow

# OpenCV Default config file for face detection
haar_file = '/content/drive/My Drive/Colab Notebooks/Config/haarcascade_frontalface_default.xml'
  
# Root Folder used to store the face data images
datasets = '/content/drive/My Drive/Colab Notebooks/Image Datasets'
 
# Subfolder used to store my images (to be changed with your name)
sub_data = 'Carlos'     
  
path = os.path.join(datasets, sub_data) 
if not os.path.isdir(path): 
    os.mkdir(path) 
  
# define the size of images  
(width, height) = (130, 100)     
  
# Analyse the video training file 
face_cascade = cv2.CascadeClassifier(haar_file) 
webcam = cv2.VideoCapture(video_file_train)  
  
# Capture the first 30 images of the train video
count = 1
while count < 30: 
    if not webcam.isOpened():
        print('Unable to load the video file.')
        sleep(5)
        pass  
    (_, im) = webcam.read() 
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY) 
    faces = face_cascade.detectMultiScale(gray, 1.3, 4) 
    for (x, y, w, h) in faces: 
        cv2.rectangle(im, (x, y), (x + w, y + h), (255, 0, 0), 2) 
        face = gray[y:y + h, x:x + w] 
        face_resize = cv2.resize(face, (width, height)) 
        cv2.imwrite('% s/% s.png' % (path, count), face_resize) 
    count += 1

Las siguientes lineas de codigo entrenan con las imagenes extraidas del video de entrenamiento e intentan reconocer la cara de la persona en el video de prueba. 

Primero, se analiza la carpeta "Image Dataset" y sus subcarpetas para crear una lista de imagenes almacenadas y su nombre correspondiente. Luego, OpenCV entrena con esta lista de imagenes y sus nombres correspondientes. Finalmente, *CascadeClassifier* se usa para detectar la cara. 

Posteriormente, los primeros 30 cuadros del video de prueba se utilizan para la deteccion de rostros. Cada uno de ellos es analizado con los metodos *detectMultiScale()* y *predict()* para reconocer la cara. El nombre de la persona detectada se muestra con un rectangulo y un indicador de confianza de prediccion. 

In [0]:
# User OpenCV Face Recognizer to train the AI & Cascade Classifier to detect faces
  
# Create a list of images and a list of corresponding names 
(images, lables, names, id) = ([], [], {}, 0) 
for (subdirs, dirs, files) in os.walk(datasets): 
    for subdir in dirs: 
        names[id] = subdir 
        subjectpath = os.path.join(datasets, subdir) 
        for filename in os.listdir(subjectpath): 
            path = subjectpath + '/' + filename 
            lable = id
            images.append(cv2.imread(path, 0)) 
            lables.append(int(lable)) 
        id += 1
(width, height) = (130, 100) 
  
# Create a Numpy array from the two lists above 
(images, lables) = [numpy.array(lis) for lis in [images, lables]] 
  
# Train the model from the images with OpenCV Face Recognizer
model = cv2.face.LBPHFaceRecognizer_create()
model.train(images, lables) 

#print(lables, names, id)

# Use OpenCV Cascade Classifier to detect faces 
face_cascade = cv2.CascadeClassifier(haar_file) 
webcam = cv2.VideoCapture(video_file_test)

count = 1
while count < 30:
    (_, im) = webcam.read() 
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY) 
    faces = face_cascade.detectMultiScale(gray, 1.3, 5) 
    for (x, y, w, h) in faces: 
        cv2.rectangle(im, (x, y), (x + w, y + h), (255, 0, 0), 2) 
        face = gray[y:y + h, x:x + w] 
        face_resize = cv2.resize(face, (width, height)) 
        # Try to recognize the face 
        prediction = model.predict(face_resize) 
        print(prediction)
        cv2.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 3) 
  
        if prediction[1]<100: 
           cv2.putText(im, '% s - %.0f' % (names[prediction[0]], prediction[1]), (x-10, y-10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0)) 
           cv2_imshow(im)
           break
        else: 
          cv2.putText(im, 'not recognized', (x-10, y-10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0)) 
  
    count += 1